In [4]:
import numpy as np
import pandas as pd

In [5]:
df = pd.read_csv("Data/raw_data/extracted_files/bank_churners.csv")
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
import warnings
warnings.filterwarnings("ignore")

In [3]:
X_train = pd.read_csv("Data/processed_data/X_train.csv")
X_val = pd.read_csv("Data/processed_data/X_val.csv")
X_test = pd.read_csv("Data/processed_data/X_test.csv")
y_train = pd.read_csv("Data/processed_data/y_train.csv")
y_val = pd.read_csv("Data/processed_data/y_val.csv")
y_test = pd.read_csv("Data/processed_data/y_test.csv")

X_test.head()

,ord__Card_Category,ord__Education_Level,ord__Income_Category,ohe__Gender_F,ohe__Gender_M,ohe__Marital_Status_Divorced,ohe__Marital_Status_Married,ohe__Marital_Status_Single,ohe__Marital_Status_Unknown,remainder__Customer_Age,remainder__Dependent_count,remainder__Months_on_book,remainder__Total_Relationship_Count,remainder__Months_Inactive_12_mon,remainder__Contacts_Count_12_mon,remainder__Credit_Limit,remainder__Total_Revolving_Bal,remainder__Avg_Open_To_Buy,remainder__Total_Amt_Chng_Q4_Q1,remainder__Total_Trans_Amt,remainder__Total_Trans_Ct,remainder__Total_Ct_Chng_Q4_Q1,remainder__Avg_Utilization_Ratio
0,-0.256861,0.827738,-0.747350,0.953358,-0.953358,-0.281458,1.082741,-0.802736,-0.283496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.256861,-1.505519,-0.747350,0.953358,-0.953358,-0.281458,-0.923582,1.245740,-0.283496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.256861,0.827738,1.957982,-1.048924,1.048924,-0.281458,-0.923582,-0.802736,3.527390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.256861,-1.505519,-0.071017,0.953358,-0.953358,3.552931,-0.923582,-0.802736,-0.283496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.256861,0.827738,-0.747350,0.953358,-0.953358,-0.281458,-0.923582,1.245740,-0.283496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from typing import List
from Model_Utils.feature_nan_imputation import DataImputer
from Model_Utils.feature_outlier_handling import OutlierHandler

In [11]:
df = pd.read_csv("Data/raw_data/extracted_files/bank_churners.csv")
X = df.drop(columns=["Attrition_Flag"])
y = df[["Attrition_Flag"]]
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

In [13]:
imputer = DataImputer(numeric_strategy="mean", categorical_strategy="mode")
X_train_impute = imputer.fit_transform(X_train)
y_train_impute = imputer.fit_transform(y_train)
outlier_handler = OutlierHandler(iqr_threshold=1.5)
X_train_transformed = outlier_handler.fit_transform(X_train_impute,'yeo')

No numeric columns to impute.


TypeError: OutlierHandler.__init__() missing 1 required positional argument: 'df'

In [7]:

X = df.drop(columns=["Attrition_Flag"])
y = df[[]"Attrition_Flag"]]
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)
ordinal_cols= ["Card_Category", "Education_Level", "Income_Category"]
ohe_cols = ["Gender", "Marital_Status"]
categories: dict = {
    "Education_Level": ["Unknown", "Uneducated", "High School", "College", "Graduate", "Post-Graduate", "Doctorate"],
    "Income_Category": ["Unknown", "Less than $40K", "$40K - $60K", "$60K - $80K", "$80K - $120K", "$120K +"],
    "Card_Category": ["Blue", "Silver", "Gold", "Platinum"]
}

encoding_transformer = ColumnTransformer([
            ('ord', OrdinalEncoder(categories=[categories[col] for col in ordinal_cols]), ordinal_cols),
            ('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), ohe_cols)
        ], remainder='passthrough')


full_pipeline = Pipeline([
            ('encode', encoding_transformer),
            ('scale', StandardScaler())
        ])

# Fit-transform and get feature names
X_train_final = full_pipeline.fit_transform(X_train)
X_val_final = full_pipeline.transform(X_val)
X_test_final = full_pipeline.transform(X_test)
feature_names = full_pipeline.get_feature_names_out()

SyntaxError: unmatched ']' (4282863476.py, line 2)

In [24]:
df1=pd.DataFrame(X_train_final, columns=feature_names)
df2=pd.DataFrame(X_val_final, columns=feature_names)
df3=pd.DataFrame(X_test_final, columns=feature_names)

In [28]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2026 entries, 0 to 2025
Data columns (total 24 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ord__Card_Category                   2026 non-null   float64
 1   ord__Education_Level                 2026 non-null   float64
 2   ord__Income_Category                 2026 non-null   float64
 3   ohe__Gender_F                        2026 non-null   float64
 4   ohe__Gender_M                        2026 non-null   float64
 5   ohe__Marital_Status_Divorced         2026 non-null   float64
 6   ohe__Marital_Status_Married          2026 non-null   float64
 7   ohe__Marital_Status_Single           2026 non-null   float64
 8   ohe__Marital_Status_Unknown          2026 non-null   float64
 9   remainder__CLIENTNUM                 2026 non-null   float64
 10  remainder__Customer_Age              2026 non-null   float64
 11  remainder__Dependent_count    